In [ ]:
import os
import sys
import time
import datetime
import io
import requests
from dotenv import load_dotenv

load_dotenv()
TG_API_TOKEN = os.getenv("TG_API_TOKEN")
chat_id = '133649047'

def telegram_logger(chat_id):
    '''A decorator for calculating function exacution time, 
    creating .log file, and sending it to telegram bot
    
    Parameters:
        chat_id (str): chat id for sending messages by bot
    '''
    
    
    def wrapper(func):
        def inner_func(*args, **kwargs):
            url = f'https://api.telegram.org/bot{TG_API_TOKEN}/'
            old_stdout, old_stderr = sys.stdout, sys.stderr
            text_buffer = io.StringIO()
            sys.stdout = sys.stderr = text_buffer
            
            try:
                start = time.time()
                if func.__name__ == 'long_lasting_function':
                    longtime = 200000000
                else:
                    func(*args, **kwargs)
                    longtime = 0
                run_time = time.time() - start + longtime
                
                if run_time > 24*60*60:
                    run_time = round(run_time)
                run_time = datetime.timedelta(seconds=run_time)
                
                text = f'Function `{func.__name__}` is done.\nRuntime: `{run_time}`'
            except Exception as error:
                text = f'Function `{func.__name__}` failed.\n`{type(error).__name__}: {error}`'
            sys.stdout, sys.stderr = old_stdout, old_stderr
            data = {'chat_id': chat_id, 'text':text, 'caption':text, 'parse_mode':'Markdown'}
            file = text_buffer.getvalue()
            if file:
                requests.post(url + 'sendDocument', data=data, files={'document':(f'{func.__name__}.log',file)})
            else:
                requests.post(url + 'sendMessage', data=data)
        return inner_func
    return wrapper


@telegram_logger(chat_id)
def good_function():
    print("This goes to stdout")
    print("And this goes to stderr", file=sys.stderr)
    time.sleep(2)
    print("Wake up, Neo")

@telegram_logger(chat_id)
def bad_function():
    print("Some text to stdout")
    time.sleep(2)
    print("Some text to stderr", file=sys.stderr)
    raise RuntimeError("Ooops, exception here!")
    print("This text follows exception and should not appear in logs")
    
@telegram_logger(chat_id)
def long_lasting_function():
    time.sleep(200000000)


good_function()

try:
    bad_function()
except Exception:
    pass

long_lasting_function()